<a href="https://colab.research.google.com/github/bingcen118/darknet/blob/master/yolobingcen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!git clone https://github.com/bingcen118/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 8278 (delta 0), reused 0 (delta 0), pack-reused 8277
Receiving objects: 100% (8278/8278), 141.58 MiB | 31.88 MiB/s, done.
Resolving deltas: 100% (4519/4519), done.


In [2]:
!wget -O cuda_9.2.88_396.26_linux.run -c https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux

--2021-04-13 20:34:15--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda_9.2.88_396.26_linux
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda_9.2.88_396.26_linux [following]
--2021-04-13 20:34:15--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda_9.2.88_396.26_linux
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda_9.2.88_396.26_linux.run?_Reivc75YR6FwEkFP9gpkRjuPdF2cQWfo9XlFCBHKeEqgEG2AOX3BPHBCYbNWazf-kd1fgKwciQ31-dcHJmYriRgLsNLjokxpdvDpXp7JiMXrzKuoQJERZoLYhMEJOJI0CJFg52Sx5Ko6qWf7lfO0hVVtIlB5sOUKnChuMPSMah8_bwpHHiq764 [following]
-

In [4]:
!chmod +x cuda_9.2.88_396.26_linux.run
!./cuda_9.2.88_396.26_linux.run --verbose --silent --toolkit --override

Installing the CUDA Toolkit in /usr/local/cuda-9.2 ...
Verifying archive integrity... All good.

Uncompressing NVIDIA CUDA..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [9]:
import os
os.environ['PATH'] += ':/usr/local/cuda-9.2/bin'

In [21]:
cd darknet

/root/darknet


In [22]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -W

In [23]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-04-13 20:42:34--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  82.6MB/s    in 2.9s    

2021-04-13 20:42:37 (82.6 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [24]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

In [25]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-13 20:43:11--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  66.0MB/s    in 2.3s    

2021-04-13 20:43:14 (66.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [26]:
!mkdir hat_backup

In [27]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector train cfg/multi.data cfg/yolov3hat.cfg darknet53.conv.74

流式输出内容被截断，只能显示最后 5000 行内容。
1709: 2.091564, 2.331085 avg, 0.001000 rate, 0.013295 seconds, 1709 images
Loaded: 0.011193 seconds
1710: 2.835874, 2.381564 avg, 0.001000 rate, 0.013913 seconds, 1710 images
Resizing
608
Loaded: 0.000045 seconds
1711: 2.211419, 2.364549 avg, 0.001000 rate, 0.015425 seconds, 1711 images
Loaded: 0.012445 seconds
1712: 1.006729, 2.228767 avg, 0.001000 rate, 0.013812 seconds, 1712 images
Loaded: 0.013065 seconds
1713: 3.490297, 2.354920 avg, 0.001000 rate, 0.016584 seconds, 1713 images
Loaded: 0.010445 seconds
1714: 2.959419, 2.415370 avg, 0.001000 rate, 0.019452 seconds, 1714 images
Loaded: 0.008869 seconds
1715: 0.941522, 2.267985 avg, 0.001000 rate, 0.028951 seconds, 1715 images
Loaded: 0.005163 seconds
1716: 1.398036, 2.180990 avg, 0.001000 rate, 0.021576 seconds, 1716 images
Loaded: 0.004587 seconds
1717: 2.436169, 2.206508 avg, 0.001000 rate, 0.022689 seconds, 1717 images
Loaded: 0.007234 seconds
1718: 1.586179, 2.144475 avg, 0.001000 rate, 0.020448 second

In [29]:
!ls -hlt hat_backup/

total 331M
-rw-r--r-- 1 root root 34M Apr 13 20:46 yolov3hat.backup
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_900.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_800.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_700.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_600.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_500.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_400.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_300.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_200.weights
-rw-r--r-- 1 root root 34M Apr 13 20:45 yolov3hat_100.weights


In [30]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.2/lib64 && ./darknet detector test cfg/multi.data cfg/yolov3hat.cfg hat_backup/yolov3hat.backup data_folder/testset/frame0030.jpg -thresh .50

layer     filters    size              input                output
    0 conv     16  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  16  0.150 BFLOPs
    1 max          2 x 2 / 2   416 x 416 x  16   ->   208 x 208 x  16
    2 conv     32  3 x 3 / 1   208 x 208 x  16   ->   208 x 208 x  32  0.399 BFLOPs
    3 max          2 x 2 / 2   208 x 208 x  32   ->   104 x 104 x  32
    4 conv     64  3 x 3 / 1   104 x 104 x  32   ->   104 x 104 x  64  0.399 BFLOPs
    5 max          2 x 2 / 2   104 x 104 x  64   ->    52 x  52 x  64
    6 conv    128  3 x 3 / 1    52 x  52 x  64   ->    52 x  52 x 128  0.399 BFLOPs
    7 max          2 x 2 / 2    52 x  52 x 128   ->    26 x  26 x 128
    8 conv    256  3 x 3 / 1    26 x  26 x 128   ->    26 x  26 x 256  0.399 BFLOPs
    9 max          2 x 2 / 2    26 x  26 x 256   ->    13 x  13 x 256
   10 conv    512  3 x 3 / 1    13 x  13 x 256   ->    13 x  13 x 512  0.399 BFLOPs
   11 max          2 x 2 / 1    13 x  13 x 512   ->    13 x  13 x 512
   12 con

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [38]:
!cp hat_backup/yolov3hat.backup /content/gdrive/My\ Drive/tiny-yolo-hat.weights